In [3]:
!pip install --upgrade imutils
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 19.1 MB/s eta 0:00:00


In [ ]:
# import cv2
# from matplotlib import pyplot as plt
# import numpy as np
# import imutils
# import easyocr

# def predict_license_plate(image_path):
#     # Load and preprocess the image
#     img = cv2.imread(image_path)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     bfilter = cv2.bilateralFilter(img, 11, 17, 17)  # Noise reduction
#     edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

#     # Find contours
#     keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     contours = imutils.grab_contours(keypoints)
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
#     location = None

#     # Find the contour with 4 points (assumed to be the license plate)
#     for contour in contours:
#         approx = cv2.approxPolyDP(contour, 10, True)
#         if len(approx) == 4:
#             location = approx
#             break

#     # Masking the license plate area
#     if location is not None:
#         mask = np.zeros(gray.shape, np.uint8)
#         new_image = cv2.drawContours(mask, [location], 0, 255, -1)
#         new_image = cv2.bitwise_and(img, img, mask=mask)

#         # Extract the license plate region
#         (x, y) = np.where(mask == 255)
#         (x1, y1) = (np.min(x), np.min(y))
#         (x2, y2) = (np.max(x), np.max(y))
#         cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

#         # Use EasyOCR to read the text
#         reader = easyocr.Reader(['en'])
#         result = reader.readtext(cropped_image)

#         # Get the recognized text
#         if result:
#             text = result[0][-2]  # Extract the text from the result
#             font = cv2.FONT_HERSHEY_SIMPLEX
#             # Draw the detected plate area and text on the image
#             res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1] + 60),
#                               fontFace=font, fontScale=1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
#             res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0, 255, 0), 3)

#             # Display the final image with the license plate and text
#             plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
#             plt.show()

#             return text
#         else:
#             return "No text detected on license plate"
#     else:
#         return "License plate area not found"


In [ ]:
!pip install pyngrok

from pyngrok import ngrok

# Replace 'YOUR_AUTH_TOKEN' with your actual ngrok auth token
!ngrok config add-authtoken '2PrRQ7oJylJo1q5mw96enFFWtuO_2oNEZXV3uVkAZFujSpbC'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
# Install necessary packages
!pip install flask opencv-python-headless numpy imutils easyocr pyngrok
!pip install --upgrade requests urllib3


import os
import cv2
import numpy as np
import imutils
import easyocr
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def predict_license_plate(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return "Invalid image path or file"

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(bfilter, 30, 200)

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    location = None

    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

        reader = easyocr.Reader(['en'])
        result = reader.readtext(cropped_image)

        if result:
            text = result[0][-2]
            return text
        else:
            return "No text detected"
    else:
        return "License plate area not found"

@app.route('/upload', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    filename = secure_filename(file.filename)
    filepath = os.path.join(UPLOAD_FOLDER, filename)
    file.save(filepath)

    predicted_text = predict_license_plate(filepath)
    return jsonify({'predicted_number_plate': predicted_text})

# Set up ngrok
port = 5000
public_url = ngrok.connect(port)
print(f" * ngrok tunnel available at: {public_url}")

# Run the Flask server
app.run(port=port)


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_in

KeyboardInterrupt: 

In [ ]:
!kill $(pgrep ngrok)  # Stop any running ngrok processes
      # Restart the tunnel


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [ ]:
!ngrok http 5000


Traceback (most recent call last):
  File "/usr/lib/python3.10/subprocess.py", line 1209, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1959, in _wait
    (pid, sts) = self._try_wait(0)
  File "/usr/lib/python3.10/subprocess.py", line 1917, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/ngrok.py", line 582, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/ngrok.py", line 570, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 276, in run_process
    subprocess.call(start)
  File "/usr/lib/python3.10/subprocess.py", line 347, in call
    return p.wait(ti

In [4]:
!pip install pyngrok

from pyngrok import ngrok

# Replace 'YOUR_AUTH_TOKEN' with your actual ngrok auth token
!ngrok config add-authtoken '2PrRQ7oJylJo1q5mw96enFFWtuO_2oNEZXV3uVkAZFujSpbC'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from flask import Flask, request, jsonify
import os
import cv2
import numpy as np
import imutils
import easyocr
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Set up upload folder
UPLOAD_FOLDER = './uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Function to predict license plate
def predict_license_plate(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return "Invalid image path or file"

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(bfilter, 30, 200)

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    location = None

    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

        reader = easyocr.Reader(['en'])
        result = reader.readtext(cropped_image)

        if result:
            text = result[0][-2]
            return text
        else:
            return "No text detected"
    else:
        return "License plate area not found"

# API endpoint to handle the upload and process the image
@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    filename = secure_filename(file.filename)
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(file_path)

    try:
        predicted_text = predict_license_plate(file_path)
        return jsonify({'plate_text': predicted_text})
    except Exception as e:
        return jsonify({'error': f'Error during prediction: {str(e)}'}), 500

# if __name__ == '__main__':
#     # Set up ngrok to expose the Colab server
#     from flask_ngrok import run_with_ngrok
#     run_with_ngrok(app)
#     app.run()
# Set up ngrok
port = 5000
public_url = ngrok.connect(port)
print(f" * ngrok tunnel available at: {public_url}")

# Run the Flask server
app.run(port=port)

 * ngrok tunnel available at: NgrokTunnel: "https://1611-34-29-122-143.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
